In [9]:
import numpy as np
import random

In [10]:
num_vals = 10
generated_probs = np.random.rand(num_vals)
norm_probs = generated_probs/sum(generated_probs)

In [44]:
class AliasedRandomizer:
    """Class that implements the Vose's Alias Method as explained in 
    https://www.keithschwarz.com/darts-dice-coins/
    """
    def __init__(self, probabilities) -> None:
        self.num_vals = len(probabilities)
        self.alias = [(0, None) for _ in range(self.num_vals)]
        self.prob = np.zeros(self.num_vals)

        self.small = []
        self.large = []

        scaled_probabilities = probabilities * self.num_vals

        for idx, prob_i in enumerate(scaled_probabilities):
            if prob_i < 1:
                self.small.append((idx, prob_i))
            else:
                self.large.append((idx, prob_i))
                
        while len(self.small) > 0 and len(self.large) > 0:
            lower, prob_lower = self.small.pop(0)
            greater, prob_greater = self.large.pop(0)

            self.prob[lower] = prob_lower
            self.alias[lower] = (prob_lower, greater)

            prob_greater = (prob_greater + prob_lower) - 1
            if prob_greater < 1:
                self.small.append((greater, prob_greater))
            else:
                self.large.append((greater,prob_greater))
                
        while len(self.large) > 0:
            greater, prob_greater = self.large.pop(0)
            self.prob[greater] = 1
        while len(self.small) > 0:
            lower, prob_lower = self.small.pop(0)
            self.prob[lower] = 1
            
    def sample(self):
        this_pr = random.random()*self.num_vals
        idx = int(this_pr)
        pr, alias = self.alias[idx]
        
        if (this_pr - idx) > pr:
            return int(idx)
        else:
            return int(alias)

In [45]:
mar  = AliasedRandomizer(norm_probs)

In [46]:
num_random_weights = [random.randint(1, 100) for _ in range(1000)]

normalized_probabilities = {}

for idx, node_neighbors in enumerate(num_random_weights):    
    num_vals = node_neighbors
    generated_probs = np.random.rand(num_vals)
    norm_probs = generated_probs/sum(generated_probs)
    normalized_probabilities[idx] = norm_probs

In [52]:
%%prun -r
for idx, norm_probs in normalized_probabilities.items():
    mar  = AliasedRandomizer(norm_probs)
    for r in range(1000):
        mar.sample()

         2304431 function calls in 1.377 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1000000    0.663    0.000    0.702    0.000 495210155.py:37(sample)
        1    0.531    0.531    1.377    1.377 <string>:1(<module>)
     1000    0.120    0.000    0.144    0.000 495210155.py:2(__init__)
  1000000    0.038    0.000    0.038    0.000 {method 'random' of '_random.Random' objects}
    98980    0.010    0.000    0.010    0.000 {method 'pop' of 'list' objects}
    98980    0.005    0.000    0.005    0.000 {method 'append' of 'list' objects}
   103467    0.004    0.000    0.004    0.000 {built-in method builtins.len}
     1000    0.003    0.000    0.003    0.000 495210155.py:4(<listcomp>)
     1000    0.002    0.000    0.002    0.000 {built-in method numpy.zeros}
        1    0.000    0.000    1.377    1.377 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'items' of 'dict' objects}
  

In [16]:
def prepare_aliased_randomizer(neighbor_names, weights):
    '''Implemented according to the alias method.

    :param weights:
    :return: Aliased randomizer
    '''
    N = len(weights)
    if N == 0:
        raise ValueError('Node has no neighbors. Check the input dataset.')
    avg = sum(weights) / N
    aliases = [(1, None)] * N
    smalls = ((i, w / avg) for i, w in enumerate(weights) if w < avg)
    bigs = ((i, w / avg) for i, w in enumerate(weights) if w >= avg)
    small, big = next(smalls, None), next(bigs, None)
    while big and small:
        aliases[small[0]] = (small[1], big[0])
        big = (big[0], big[1] - (1 - small[1]))
        if big[1] < 1:
            small = big
            big = next(bigs, None)
        else:
            small = next(smalls, None)

    def weighted_random():
        r = random.random() * N
        i = int(r)
        odds, alias = aliases[i]
        if (r - i) > odds:
            return neighbor_names[alias]
        else:
            return neighbor_names[i]
        # return alias if (r - i) > odds else i

    return weighted_random


In [18]:
%%timeit -n 1000
neighbor_names = generated_probs
wr = prepare_aliased_randomizer(neighbor_names, norm_probs)
for r in range(10000):
    wr()

4.01 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
